# Chain of Responsibility Method Design Pattern:

## Some Use Cases:
1. Data Validation Pipeline:
- Use Case: In data engineering, data often needs to pass through a series of validation checks (e.g., format validation, completeness, consistency). The Chain of Responsibility pattern can be used to process data through multiple validation steps, where each step decides if the data passes or needs further checks.
- Benefit: This pattern allows the validation logic to be modular, easy to extend, and flexible. Each validation step can be added or modified independently without changing the entire validation pipeline.
2. Error Handling in Data Processing:
- Use Case: In complex data processing pipelines, errors or exceptions may need to be handled in different ways based on the type of error (e.g., data format error, missing value, network timeout). The Chain of Responsibility pattern allows for the delegation of error handling to different handlers based on the error type.
- Benefit: The pattern enables flexible error handling by allowing multiple handlers to process an error in a chain, ensuring that the most appropriate handler is chosen dynamically.
3. Log Processing:
- Use Case: When processing logs from multiple data sources (e.g., system logs, application logs, sensor data), different log levels (INFO, WARNING, ERROR) or types of logs may need to be processed differently. The Chain of Responsibility pattern can be used to pass logs through a series of processors based on their severity or type.
- Benefit: This approach ensures that logs are handled and processed in an organized manner, allowing for easy extension of log processors (e.g., filtering, aggregation, or alerting) without changing the core log processing logic.

## 1. Scenario: Logging System
- In a logging system, we need to handle different log levels (e.g., DEBUG, INFO, WARN, ERROR) and process them in a sequence. The system should allow for different levels of logging to be handled differently based on priority. For example, ERROR logs should be immediately handled, while INFO or DEBUG logs may be stored for later analysis.

#### Tasks:
- For each log level (DEBUG, INFO, WARN, ERROR), we need different processing (e.g., write to file, print to console, etc.).
- We want to dynamically change the processing order or handlers.

### 1.1 Using Traditional Approach:
- Using traditional conditional checks like if-else or switch-case.

In [1]:
class Logger:
    def log(self, level, message):
        if level == "DEBUG":
            self.log_debug(message)
        elif level == "INFO":
            self.log_info(message)
        elif level == "WARN":
            self.log_warn(message)
        elif level == "ERROR":
            self.log_error(message)
    
    def log_debug(self, message):
        print(f"DEBUG: {message}")
    
    def log_info(self, message):
        print(f"INFO: {message}")
    
    def log_warn(self, message):
        print(f"WARN: {message}")
    
    def log_error(self, message):
        print(f"ERROR: {message}")

# Usage
logger = Logger()
logger.log("DEBUG", "This is a debug message")


DEBUG: This is a debug message


### Issues with the Traditional Approach:
- Tightly Coupled: Every new log level requires modifying the log method, violating the Open/Closed Principle.
- Hard to Maintain: If new logging levels are introduced or the behavior for an existing level changes, it forces us to modify the existing logic, leading to a potential for errors and bugs.
- Lack of Flexibility: You cannot change the flow of logging dynamically (e.g., change the order of log handling).

### 1.2 Using State Method Pattern:
- With State Pattern, we can define states like DEBUG, INFO, WARN, and ERROR as separate classes. However, State Pattern comes with its own limitations in this case.
### Issues Solved by State Pattern:
- Separation of Concerns: Each log level is encapsulated in its own class, which decouples the logging logic for each level and adheres to the Single Responsibility Principle.
- Extensibility: New log levels can be added without modifying the Logger class, addressing the issue of Open/Closed Principle.


In [8]:
# Context
class Logger:
    def __init__(self, level: LogLevel):
        self.level = level
    
    def log(self, message):
        self.level.handle(message)

# State
class LogLevel:
    def handle(self, message):
        pass

# Concrete State 1
class DebugLogLevel(LogLevel):
    def handle(self, message):
        print(f"DEBUG: {message}")

# Concrete State 2
class InfoLogLevel(LogLevel):
    def handle(self, message):
        print(f"INFO: {message}")

# Concrete State 3
class WarnLogLevel(LogLevel):
    def handle(self, message):
        print(f"WARN: {message}")

# Concrete State 4
class ErrorLogLevel(LogLevel):
    def handle(self, message):
        print(f"ERROR: {message}")

# Usage
logger = Logger(DebugLogLevel())
logger.log("This is a debug message")


DEBUG: This is a debug message


### Issues with State Pattern:
- Fixed Transitions: The state transitions are fixed. Once you set a state (e.g., DEBUG), you cannot easily change the order or flow of log levels (e.g., processing ERROR before INFO).
- Rigid Structure: Each log level requires a separate class, leading to a rigid structure where adding new levels requires new classes and changes to the Logger class.
- No Dynamic Behavior: The flow of log handling is not dynamic. The state is fixed, and you cannot change the handling order dynamically (like handling ERROR before INFO).

### 1.3 Solve with Strategy Pattern
The Strategy Pattern provides a flexible way to change the logging strategy (e.g., log to console, log to file) but still doesn't solve the issue of dynamic ordering of log levels.

### Issues Solved by Strategy Pattern:
- Dynamic Strategy Change: Allows you to dynamically change the logging strategy at runtime (e.g., from console to file logging), solving the rigidity of the previous approaches.
- Separation of Concerns: The logging logic is separated into different strategies (e.g., ConsoleLogStrategy, FileLogStrategy), following the Single Responsibility Principle.

In [7]:
# Context
class Logger:
    def __init__(self, strategy: LogStrategy):
        self.strategy = strategy
    
    def log(self, message):
        self.strategy.log(message)

# Strategy
class LogStrategy:
    def log(self, message):
        pass

# Concrete Strategy 1
class ConsoleLogStrategy(LogStrategy):
    def log(self, message):
        print(f"Console Log: {message}")

# Concrete Strategy 2
class FileLogStrategy(LogStrategy):
    def log(self, message):
        with open("log.txt", "a") as file:
            file.write(f"File Log: {message}\n")

# Usage
logger = Logger(ConsoleLogStrategy())
logger.log("This is a console log message")


Console Log: This is a console log message


### Issues with Strategy Pattern:
- No Flow Control: The strategy defines how to handle logs but doesn't define when to handle them. You can't change the order or manage the flow of log levels (e.g., process ERROR first, then INFO).
- No Sequential Handling: Unlike the Chain of Responsibility, the Strategy Pattern lacks the ability to process logs sequentially (i.e., define the order of log handling).
- No Dynamic Log Level Handling: The pattern is effective for selecting how to handle logs but doesn't manage the different levels of logs dynamically or chain them in sequence.

### 1.4 Using Chain of Responsibility Method Pattern:
The Chain of Responsibility pattern provides a solution to dynamically change the order in which log levels are handled, enabling the sequential processing of logs.


### Components of the Chain of Responsibility pattern:

- Handler: Defines the interface for handling requests and optionally passing them to the next handler.
- ConcreteHandler: Implements the Handler interface and processes the request. If it can't handle the request, it passes it along the chain.
- Client: Initiates the request and sends it to the first handler in the chain.

#### Chain:
- The linked structure of handlers, where each handler knows the next one in the chain to forward the request if it cannot process it.

In [6]:
# Handler (Defines the interface for handling requests)
class LogHandler:
    def __init__(self, next_handler=None):
        self.next_handler = next_handler  # Chain link to next handler
    
    def handle(self, level, message):
        if self.next_handler:
            self.next_handler.handle(level, message)  # Pass to next handler

# ConcreteHandler for DEBUG logs
class DebugLogHandler(LogHandler):
    def handle(self, level, message):
        if level == "DEBUG":
            print(f"DEBUG: {message}")
        elif self.next_handler:
            self.next_handler.handle(level, message)  # Pass to next handler if not handled

# ConcreteHandler for INFO logs
class InfoLogHandler(LogHandler):
    def handle(self, level, message):
        if level == "INFO":
            print(f"INFO: {message}")
        elif self.next_handler:
            self.next_handler.handle(level, message)  # Pass to next handler if not handled

# ConcreteHandler for WARN logs
class WarnLogHandler(LogHandler):
    def handle(self, level, message):
        if level == "WARN":
            print(f"WARN: {message}")
        elif self.next_handler:
            self.next_handler.handle(level, message)  # Pass to next handler if not handled

# ConcreteHandler for ERROR logs
class ErrorLogHandler(LogHandler):
    def handle(self, level, message):
        if level == "ERROR":
            print(f"ERROR: {message}")
        elif self.next_handler:
            self.next_handler.handle(level, message)  # Pass to next handler if not handled

# Client (Starts the request)
# Chain of handlers, each linked to the next one
handler_chain = DebugLogHandler(
    InfoLogHandler(
        WarnLogHandler(
            ErrorLogHandler()
        )
    )
)

# Logs get passed through the chain
handler_chain.handle("DEBUG", "This is a debug message")  # DEBUG: This is a debug message
handler_chain.handle("INFO", "This is an info message")  # INFO: This is an info message
handler_chain.handle("ERROR", "This is an error message")  # ERROR: This is an error message


DEBUG: This is a debug message
INFO: This is an info message
ERROR: This is an error message


### How Chain of Responsibility Solves the Issues:
- Dynamic Handling: The chain can be dynamically modified. You can add, remove, or reorder handlers without modifying the existing code, providing flexibility in log handling.
- Decoupled Process: Each handler is decoupled from others. They are responsible for handling their specific log level and delegate the rest to the next handler, promoting Single Responsibility Principle.
- Flexible Log Level Flow: The order in which log levels are handled can be modified dynamically. For example, you can handle ERROR first and then INFO by changing the order of handlers in the chain.
- Extensibility: New log levels can be added easily by creating a new handler, without changing existing code or breaking the existing flow.

### Summary of Issue Resolutions:
1.1 Traditional Method:
- Issues: Tightly coupled, hard to extend, no dynamic flexibility.

1.2 State Pattern:
- Solved by State Pattern: Separation of concerns, more extensibility, but fixed flow and rigid structure.
- Issues: Fixed state transitions, rigid structure, no dynamic flow.

1.3 Strategy Pattern:
- Solved by Strategy Pattern: Allows dynamic strategy changes but still lacks flow control and sequential handling.
- Issues: No flow control, no sequential handling, no dynamic log level handling.

1.4 Chain of Responsibility:
- Solved by Chain of Responsibility: Enables dynamic flow control, decoupled handling, and flexible ordering of log levels.
- Clear Advantage: Dynamic and flexible log handling, decoupled handlers, easy to add new levels or change the flow without affecting existing code.

## Changing the Order of Execution:
#### Original Order (DEBUG -> INFO -> WARN -> ERROR):

handler_chain = 
        DebugLogHandler(
    
    InfoLogHandler(
        WarnLogHandler(
            ErrorLogHandler()
        )
    )
)


#### Changing the Order (ERROR -> DEBUG -> INFO -> WARN):
handler_chain = ErrorLogHandler(
    
    DebugLogHandler(
        InfoLogHandler(
            WarnLogHandler()
        )
    )
)


In [9]:
# Handler (Defines the interface for handling requests)
class LogHandler:
    def __init__(self, next_handler=None):
        self.next_handler = next_handler  # Chain link to next handler
    
    def handle(self, level, message):
        if self.next_handler:
            self.next_handler.handle(level, message)  # Pass to next handler

# ConcreteHandler for DEBUG logs
class DebugLogHandler(LogHandler):
    def handle(self, level, message):
        if level == "DEBUG":
            print(f"DEBUG: {message}")
        elif self.next_handler:
            self.next_handler.handle(level, message)  # Pass to next handler if not handled

# ConcreteHandler for INFO logs
class InfoLogHandler(LogHandler):
    def handle(self, level, message):
        if level == "INFO":
            print(f"INFO: {message}")
        elif self.next_handler:
            self.next_handler.handle(level, message)  # Pass to next handler if not handled

# ConcreteHandler for WARN logs
class WarnLogHandler(LogHandler):
    def handle(self, level, message):
        if level == "WARN":
            print(f"WARN: {message}")
        elif self.next_handler:
            self.next_handler.handle(level, message)  # Pass to next handler if not handled

# ConcreteHandler for ERROR logs
class ErrorLogHandler(LogHandler):
    def handle(self, level, message):
        if level == "ERROR":
            print(f"ERROR: {message}")
        elif self.next_handler:
            self.next_handler.handle(level, message)  # Pass to next handler if not handled

# Client (Starts the request)
# Chain of handlers, each linked to the next one
# Chain of handlers in changed order: ERROR -> DEBUG -> INFO -> WARN
handler_chain = ErrorLogHandler(
    DebugLogHandler(
        InfoLogHandler(
            WarnLogHandler()
        )
    )
)

# Logs get passed through the chain in the new order
handler_chain.handle("DEBUG", "This is a debug message")  # DEBUG: This is a debug message
handler_chain.handle("INFO", "This is an info message")  # INFO: This is an info message
handler_chain.handle("ERROR", "This is an error message")  # ERROR: This is an error message


DEBUG: This is a debug message
INFO: This is an info message
ERROR: This is an error message
